In [14]:
import math
import sys

MAX_CLUSTERS = 5
VEC_LEN = 7
INPUT_PATTERNS =7
INPUT_TESTS = 6
DECAY_RATE = 0.5
MIN_ALPHA = 0.01
RADIUS_REDUCTION_POINT = 0.023

In [15]:
w = [[0.2, 0.6, 0.5, 0.9, 0.4, 0.2, 0.8],
     [0.9, 0.3, 0.6, 0.4, 0.5, 0.6, 0.3],
     [0.8, 0.5, 0.7, 0.2, 0.6, 0.9, 0.5],
     [0.6, 0.4, 0.2, 0.3, 0.7, 0.2, 0.4],
     [0.8, 0.9, 0.7, 0.9, 0.3, 0.2, 0.5]]

pattern = [[1, 1, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 1, 1],
           [0, 0, 1, 1, 1, 0, 0],
           [0, 0, 0, 0, 0, 0, 1],
           [1, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 1, 0, 0, 0],
           [1, 0, 1, 0, 1, 0, 1]]


tests = [[1, 1, 1, 1, 0, 0, 0],
         [0, 1, 1, 0, 1, 1, 1],
         [0, 1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 1, 1, 1, 1]]

In [16]:
class SOM_Class2:
    def __init__(self, vectorLength, maxClusters, numPatterns, numTests, minimumAlpha, decayRate, reductionPoint, weightArray):
        self.mVectorLen = vectorLength
        self.mMaxClusters = maxClusters
        self.mNumPatterns = numPatterns
        self.mNumTests = numTests
        self.mMinAlpha = minimumAlpha
        self.mDecayRate = decayRate
        self.mReductionPoint = reductionPoint
        self.mAlpha = 0.6
        self.d = []
        self.w = weightArray
        return
    
    
    def compute_input(self, vectorArray, vectorNumber):
        self.d = [0.0]*self.mMaxClusters
        
        for i in range(self.mMaxClusters):
            for j in range(self.mVectorLen):
                self.d[i] = self.d[i] + math.pow((self.w[i][j]-vectorArray[vectorNumber][j]),2)
#         print(self.d)
        return
        
        
#         print(vectorArray)
#         print(vectorNumber)
            
    def get_minimum(self,nodeArray):                    #NodeArray holding the distances of selected instance with each node
        minimum = 0
        foundNewMinimum = False
        done = False
        
        while not done:
            foundNewMinimum = False
            for i in range(self.mMaxClusters):
                if i!=minimum:
                    if nodeArray[i] < nodeArray[minimum]:
                        minimum = i
                        foundNewMinimum = True
            
            if foundNewMinimum == False:
                done = True
        
        return minimum
    
    
                
    def update_weights(self,vectorNumber, dMin, patternArray):
#         print('weights :')
#         print(self.w)
#         print('dmin')
#         print(dMin)
        for i in range(self.mVectorLen):
            #Updating winner neuron weights
            self.w[dMin][i] = self.w[dMin][i] + (self.mAlpha* (patternArray[vectorNumber][i]-self.w[dMin][i]))
            
            #Calculation(updation of weights of neighbourhood)- Only include neighbours befor radius reduction point is reached
            #Gaussian Function
            if self.mAlpha > self.mReductionPoint:
                if (dMin > 0) and (dMin < (self.mMaxClusters - 1)):
                    #updating left neighbours
                    self.w[dMin - 1][i] = self.w[dMin - 1][i] + (self.mAlpha * (patternArray[vectorNumber][i] - self.w[dMin - 1][i]))
                    # updating right neighbors
                    self.w[dMin + 1][i] = self.w[dMin + 1][i] + (self.mAlpha * (patternArray[vectorNumber][i] - self.w[dMin + 1][i]))
                else:
                    if dMin == 0:                                         #if the index of minimum is first one i.e 0 itself
                        # Update neighbor to the right.
                        self.w[dMin + 1][i] = self.w[dMin + 1][i] + (self.mAlpha * (patternArray[vectorNumber][i] - self.w[dMin + 1][i]))
                    else:                                                      #if the index of minimum is last one i.e mMaxClusters itself
                        # Update neighbor to the left. 
                        self.w[dMin - 1][i] = self.w[dMin - 1][i] + (self.mAlpha * (patternArray[vectorNumber][i] - self.w[dMin - 1][i]))
    
        return

            
        
        
    
    def training(self, patternArray):
        iterations = 0                                                         #t
        reductionFlag = False
        reductionPoint = 0
        while(self.mAlpha > self.mMinAlpha):
            iterations = iterations + 1
            for i in range(self.mNumPatterns):
                self.compute_input(patternArray, i)
                dMin = self.get_minimum(self.d)
                self.update_weights(i,dMin,patternArray)
            
            #reduction of learning rate using decay rate
            self.mAlpha = self.mDecayRate*self.mAlpha
            
            # Reducing the radius at specified point.
            if self.mAlpha < self.mReductionPoint:
                if reductionFlag == False:
                    reductionFlag = True
                    reductionPoint = iterations
                    
        print("Iterations" + str(iterations) + '\n')
        print("Neighborhood radius reduced after " + str(reductionPoint) + " iterations.\n")
        
        return
    
    
    
    def print_results(self, patternArray, testArray):
        #Printing the clusters created
        print("Clusters for training input: \n")
        for i in range(self.mNumPatterns):
            self.compute_input(patternArray,i)
            dMin = self.get_minimum(self.d)
            print("Vector (")
            for j in range(self.mVectorLen):
                print(str(patternArray[i][j]) + ", ")
            
            print(") fits into category " + str(dMin) + "\n")
                  
        
                             
        # Print weight matrix.

        print("------------------------------------------------------------------------\n")
        for i in range(self.mMaxClusters):
            print("Weights for Node " + str(i) + " connections:\n")
            print("     ")
            for j in range(self.mVectorLen):
                print("{:03.3f}".format(self.w[i][j]) + ", ")
            
            print("\n")
        
        # Print post-training tests.
        print("------------------------------------------------------------------------\n")
        print("Categorized test input:\n")
        for i in range(self.mNumTests):
            self.compute_input(testArray, i)
            
            dMin = self.get_minimum(self.d)
            
            print("Vector (")
            for j in range(self.mVectorLen):
                print(str(testArray[i][j]) + ", ")
            
            print(") fits into category " + str(dMin) + "\n")
        
        return
        
som = SOM_Class2(VEC_LEN,MAX_CLUSTERS,INPUT_PATTERNS,INPUT_TESTS,MIN_ALPHA,DECAY_RATE,RADIUS_REDUCTION_POINT,w)
som.training(pattern)
som.print_results(pattern, tests)









Iterations6

Neighborhood radius reduced after 5 iterations.

Clusters for training input: 

Vector (
1, 
1, 
1, 
0, 
0, 
0, 
0, 
) fits into category 4

Vector (
0, 
0, 
0, 
0, 
1, 
1, 
1, 
) fits into category 2

Vector (
0, 
0, 
1, 
1, 
1, 
0, 
0, 
) fits into category 0

Vector (
0, 
0, 
0, 
0, 
0, 
0, 
1, 
) fits into category 2

Vector (
1, 
0, 
0, 
0, 
0, 
0, 
0, 
) fits into category 4

Vector (
0, 
0, 
0, 
1, 
0, 
0, 
0, 
) fits into category 0

Vector (
1, 
0, 
1, 
0, 
1, 
0, 
1, 
) fits into category 2

------------------------------------------------------------------------

Weights for Node 0 connections:

     
0.009, 
0.026, 
0.415, 
0.996, 
0.410, 
0.009, 
0.035, 


Weights for Node 1 connections:

     
0.284, 
0.000, 
0.447, 
0.391, 
0.588, 
0.141, 
0.609, 


Weights for Node 2 connections:

     
0.432, 
0.004, 
0.431, 
0.002, 
0.681, 
0.259, 
0.996, 


Weights for Node 3 connections:

     
0.650, 
0.140, 
0.423, 
0.000, 
0.444, 
0.161, 
0.633, 


Weights for Node 4